In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import random
import numpy as np

# Data Scrapping (Post Text)

In [ ]:
#Choose the main site URL
site_url = 'https://forums.hardwarezone.com.sg'

def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['thread_url', 'post_text'] 
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url
    r3 = requests.get(thread_page_url)
    thread_page = r3.text
    thread_page_soup = BeautifulSoup(thread_page, 'html.parser')
    currentPage = 1
    try: #get last page
        lastPage = int(thread_page_soup.find(string=re.compile("1 of"))[8:])
    except Exception as e:
        lastPage = 1
        print(lastPage)
    
    while(currentPage <= lastPage):
        print(f'{thread_page_url} initial')
        r3 = requests.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')
        thread_page_posts = thread_page_soup.find('div', {'class': 'block-body js-replyNewMessageContainer'})

        currentPage += 1 #next page after current has been scrapped
        print(f'{currentPage} curr {lastPage} last')
        if (currentPage <= lastPage):
            thread_page_url = thread_url + 'page-' + str(currentPage)
            
        try: 
            count = 0
            for post in thread_page_posts.find_all('div', {'class': 'message-inner'}):
                post_text = ""
                try:
                    post_text = post.find('div', {'class': 'bbWrapper'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass
                row = pd.DataFrame([[thread_url, post_text]], columns=thread_cols)
                print(row)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) 
                    
        except Exception as e:
            row = pd.DataFrame([[thread_url, ""]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=False)
                
    thread_df['post_text'] = thread_df['post_text'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

    return thread_df

In [ ]:
df = pd.read_csv('urls.csv')
df = df[df['URL'].str.startswith('https://forums.hardwarezone')]

In [ ]:
for index, row in df.iterrows():
    getPosts(row['URL']).to_csv("raw_unclean.csv", mode = 'a', encoding='utf-8')

# Data Cleaning

In [4]:
df = pd.read_csv('raw_unclean.csv')

In [5]:
df.head(5)

,Unnamed: 0,thread_url,post_text
0,0.0,https://forums.hardwarezone.com.sg/threads/jus...,on grounds such as treason etc... better not u...
1,1.0,https://forums.hardwarezone.com.sg/threads/jus...,Where got so easy. Still gotch pinky at the to...
2,2.0,https://forums.hardwarezone.com.sg/threads/jus...,prusswan.net said: on grounds such as treason ...
3,3.0,https://forums.hardwarezone.com.sg/threads/jus...,prusswan.net said: on grounds such as treason ...
4,4.0,https://forums.hardwarezone.com.sg/threads/jus...,Preparing for open mob now. ...


In [6]:
df.shape

(959, 3)

In [7]:
df = df[df['post_text'].notna()]
#print(df[df['post_text'].isna()])

In [8]:
df['post_text'] = df['post_text'].astype("string")
df.dtypes

Unnamed: 0    float64
thread_url     object
post_text      string
dtype: object

In [9]:
df['subindex'] = df['post_text'].apply(lambda x: x.find("Click to expand"))
df['subindex'].mask(df['subindex'] == -1 ,-19, inplace=True)
df = df.assign(dropquotes=df.apply(lambda x: x['post_text'][x['subindex']+19:], axis=1))
df = df.drop(columns=['subindex'])

In [10]:
df['cleanascii'] = df['dropquotes'].replace(to_replace=['post_text',r'\\u[0-9A-Fa-f]{4}', r'\{\\n.*\\t\}', r'\\t|\\n|\\r', '\t|\n|\r'], value=["veryuniquestring","","veryuniquestring","",""], regex=True)



In [11]:
df.shape
#print(df[df['cleanascii'].isna()])

(947, 5)

In [12]:
df = df[df["cleanascii"].str.contains("veryuniquestring")==False]
df.shape

(907, 5)

In [13]:
df = df[df['cleanascii'].str.len()>=3]
#print(df[df['cleanascii'].str.len()<=3])

In [14]:
df = df.drop(columns=['dropquotes'])
df = df.drop(columns=['post_text'])

In [15]:
df.head(10)

,Unnamed: 0,thread_url,cleanascii
0,0.0,https://forums.hardwarezone.com.sg/threads/jus...,on grounds such as treason etc... better not u...
1,1.0,https://forums.hardwarezone.com.sg/threads/jus...,Where got so easy. Still gotch pinky at the to...
2,2.0,https://forums.hardwarezone.com.sg/threads/jus...,Dude. You know what is military chain of comma...
3,3.0,https://forums.hardwarezone.com.sg/threads/jus...,quote first
4,4.0,https://forums.hardwarezone.com.sg/threads/jus...,Preparing for open mob now. ...
5,5.0,https://forums.hardwarezone.com.sg/threads/jus...,imh pls off wifi
6,6.0,https://forums.hardwarezone.com.sg/threads/jus...,TS must be never serve army before
7,7.0,https://forums.hardwarezone.com.sg/threads/jus...,must wait till the holee emperor go first
8,8.0,https://forums.hardwarezone.com.sg/threads/jus...,myanmar junta taught us that ex-general still ...
9,9.0,https://forums.hardwarezone.com.sg/threads/jus...,TS want to tio POFMA and lim kopi ah


In [16]:
df.to_csv("cleanfakecomments.csv")